In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import io

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving test.csv to test (3).csv


### Que.1. In this dummy dataset file named **test.csv**, I have generated 4 columns with entries in **different date formats** using *rand* and *randbetween* functions in excel. All the date columns have different date formats as :

1. d1 is between 01/01/2017 and 12/31/2019
2. d2 is between the 1-Jan-20 and 31-Dec-23
3. d3 is between the values "2017-01-01 00:00:00"-"2020-08-15 23:59:59"
4. d4 is also in the same range as order_date without time stamp in the long date format, for example Friday, June 21,2019
5. I have also taken 2 other columns, namely item and price, to make sure other columns doesn't get affected by our function

Apart from NA values, there are some invalid dates also present, for instance 24th, 37th row (python index) in d1, and in 27th row in d2.

In [ ]:
# reading dataset as a dataframe
df = pd.read_csv(io.StringIO(uploaded['test.csv'].decode('utf-8')))
df.head()

,item,price,d1,d2,d3,d4
0,3,300,3/23/2019,11-May-22,4/29/2018 4:44,"Monday, January 6, 2020"
1,2,200,7/21/2018,5-May-22,4/29/2018 15:27,"Saturday, December 29, 2018"
2,2,200,2/10/2018,26-Sep-20,2/12/2017 17:34,"Monday, August 7, 2017"
3,4,400,5/9/2019,2-Jun-20,3/12/2019 3:19,"Wednesday, November 6, 2019"
4,5,500,1/16/2018,6-Jan-21,8/21/2019 17:11,"Monday, January 16, 2017"


In [ ]:
# Checking for NA values (doesn't show invalid values and invalid dates)
df.isna().sum()

item     0
price    0
d1       2
d2       0
d3       1
d4       0
dtype: int64

Detecting the columns in which dates are present and converting in to datetime64[ns] structure. Also coercing the invalid values, meaning invalid date entries will be converted to NaT (Not a Time) value. 

I haven't filled NA values or NaT values since dates can't be imputed like regular numeric values, so either left them as they are, or drop them. Since I have very few NA and NaT values, so I left them as it is. If we have significant NA or NaT values, they should be filled and corrected manually. Also the difference of dates where one value is NA or NaT, the result will be NaT.

In [ ]:
# detecting the 'Date' columns and converting it to python datetime format 
df = df.apply(lambda col: pd.to_datetime(col, errors='coerce') 
              if col.dtypes == object 
              else col, 
              axis=0)

df.dtypes

item              int64
price             int64
d1       datetime64[ns]
d2       datetime64[ns]
d3       datetime64[ns]
d4       datetime64[ns]
dtype: object

In [ ]:
# First converting datetime to string and making one date format ("%Y-%m-%d") for all date columns in df, and converting its type to datetime64[ns] again

df['d1'] = df["d1"].dt.strftime("%Y-%m-%d").astype('datetime64[ns]')
df['d2'] = df["d2"].dt.strftime("%Y-%m-%d").astype('datetime64[ns]')
df['d3'] = df["d3"].dt.strftime("%Y-%m-%d").astype('datetime64[ns]')
df['d4'] = df["d4"].dt.strftime("%Y-%m-%d").astype('datetime64[ns]')

df.head()

,item,price,d1,d2,d3,d4
0,3,300,2019-03-23,2022-05-11,2018-04-29,2020-01-06
1,2,200,2018-07-21,2022-05-05,2018-04-29,2018-12-29
2,2,200,2018-02-10,2020-09-26,2017-02-12,2017-08-07
3,4,400,2019-05-09,2020-06-02,2019-03-12,2019-11-06
4,5,500,2018-01-16,2021-01-06,2019-08-21,2017-01-16


In [ ]:
# difference of dates
df['d1-d2'] = df['d1'].sub(df['d2'], axis = 0)
df['d2-d3'] = df['d2'].sub(df['d3'], axis = 0)
df['d3-d4'] = df['d3'].sub(df['d4'], axis = 0)
df['d4-d1'] = df['d4'].sub(df['d1'], axis = 0)
df.head()

,item,price,d1,d2,d3,d4,d1-d2,d2-d3,d3-d4,d4-d1
0,3,300,2019-03-23,2022-05-11,2018-04-29,2020-01-06,-1145 days,1473 days,-617 days,289 days
1,2,200,2018-07-21,2022-05-05,2018-04-29,2018-12-29,-1384 days,1467 days,-244 days,161 days
2,2,200,2018-02-10,2020-09-26,2017-02-12,2017-08-07,-959 days,1322 days,-176 days,-187 days
3,4,400,2019-05-09,2020-06-02,2019-03-12,2019-11-06,-390 days,448 days,-239 days,181 days
4,5,500,2018-01-16,2021-01-06,2019-08-21,2017-01-16,-1086 days,504 days,947 days,-365 days


In [ ]:
# Defining a function to perform all the above steps

def date_fun(dataset):
  dataset = dataset.apply(lambda col: pd.to_datetime(col, errors = 'coerce') 
                      if col.dtypes == object
                      else col, axis = 0)
 
  dataset = dataset.apply(lambda x: dataset['x'].dt.strftime("%Y-%m-%d").astype('datetime64[ns]')
                      if list(dataset.dtypes) == 'datetime64[ns]'
                      else x, axis = 0)

  dataset[['diff1','diff2','diff3','diff4']] = dataset[['d1','d2','d3','d4']] - dataset[['d2','d3','d4','d1']].values
  return dataset()
 

This function takes column names while computing dates differences, so this is not general. This needs to be addressed.


In [ ]:
date_fun(df)

,item,price,d1,d2,d3,d4,diff1,diff2,diff3,diff4
0,3,300,2019-03-23,2022-05-11,2018-04-29,2020-01-06,-1145 days,1473 days,-617 days,289 days
1,2,200,2018-07-21,2022-05-05,2018-04-29,2018-12-29,-1384 days,1467 days,-244 days,161 days
2,2,200,2018-02-10,2020-09-26,2017-02-12,2017-08-07,-959 days,1322 days,-176 days,-187 days
3,4,400,2019-05-09,2020-06-02,2019-03-12,2019-11-06,-390 days,448 days,-239 days,181 days
4,5,500,2018-01-16,2021-01-06,2019-08-21,2017-01-16,-1086 days,504 days,947 days,-365 days


## **Que. 2.** For this problem, I have used a publicly available (kaggle and UCI ML repo) dataset called Breast Cancer Diagnostic, in which there are 31 coloumns and 569 examples

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving data.csv to data.csv


In [ ]:
data = pd.read_csv(io.StringIO(uploaded['data.csv'].decode('utf-8')))
data.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [ ]:
def correlation(dataset, threshold):
    col_corr = set() # Set of columns which are going to be deleted
    corr_matrix = dataset.corr()  # Correlation matrix
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if (corr_matrix.iloc[i, j] >= threshold) and (corr_matrix.columns[j] not in col_corr):
                colname = corr_matrix.columns[i] # getting the names of column
                col_corr.add(colname)
                if colname in dataset.columns:
                    del dataset[colname] # deleting the column from the dataset

    print(dataset)

In [ ]:
correlation(data,0.85)

           id diagnosis  ...  symmetry_worst  fractal_dimension_worst
0      842302         M  ...          0.4601                  0.11890
1      842517         M  ...          0.2750                  0.08902
2    84300903         M  ...          0.3613                  0.08758
3    84348301         M  ...          0.6638                  0.17300
4    84358402         M  ...          0.2364                  0.07678
..        ...       ...  ...             ...                      ...
564    926424         M  ...          0.2060                  0.07115
565    926682         M  ...          0.2572                  0.06637
566    926954         M  ...          0.2218                  0.07820
567    927241         M  ...          0.4087                  0.12400
568     92751         B  ...          0.2871                  0.07039

[569 rows x 21 columns]


So all the columns (10 columns) with pearson correlation greater than 0.85 have been dropped.